In [1]:
import pandas as pd

df = pd.read_csv("./data/labeled_data.csv")
cn7 = df[(df["PART_NAME"] == "CN7 W/S SIDE MLD'G RH") |(df["PART_NAME"] == "CN7 W/S SIDE MLD'G LH")]
cn7 = df.drop(columns=["_id", "TimeStamp","PART_FACT_PLAN_DATE", "Reason", "PART_FACT_SERIAL", "PART_NAME",
               "EQUIP_NAME", "EQUIP_CD", "Switch_Over_Position", "Mold_Temperature_1","Mold_Temperature_2","Mold_Temperature_5",
                "Mold_Temperature_6", "Mold_Temperature_7","Mold_Temperature_8","Mold_Temperature_9","Mold_Temperature_10",
               "Mold_Temperature_11","Mold_Temperature_12","Barrel_Temperature_7"])

In [2]:
cn7["PassOrFail"] = cn7["PassOrFail"].replace('Y',0).replace('N',1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16000\1133887326.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cn7["PassOrFail"] = cn7["PassOrFail"].replace('Y',0).replace('N',1)


In [3]:
### Clamp_Open_Position에서 눈에띄게 낮은 값들 제거
cn7 = cn7[cn7["Clamp_Open_Position"]>=200]

In [4]:
def remove_outliers(df, column):
    # 1사분위수(Q1)와 3사분위수(Q3) 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 이상치 범위 설정
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 이상치 제거
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return filtered_df

In [5]:
cn7_pass = cn7[cn7["PassOrFail"] == 0]
cn7_fail = cn7[cn7["PassOrFail"] == 1]
cn7_pass_removed = cn7_pass.copy()
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Injection_Time')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Plasticizing_Time')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Cycle_Time')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Max_Injection_Speed')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Mold_Temperature_3')

cn7_removed = pd.concat([cn7_pass_removed, cn7_fail], ignore_index=True)

In [6]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

# 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test_with_target = X_test.copy()
X_test_with_target["PassOrFail"] = y_test
X_test_with_target.to_csv("./cn7_test.csv", index=False)

# X_train, y_train 합치기
Xy_train = X_train.copy()
Xy_train["PassOrFail"] = y_train

# Average_Screw_RPM 기준으로 up만 추출하기
Xy_train = Xy_train[Xy_train["Average_Screw_RPM"] > 250]

# X_train, y_train 분리
X_train = Xy_train.drop(columns=["PassOrFail"])
y_train = Xy_train["PassOrFail"]

# 데이터 증강
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

X_resampled_with_target = X_resampled.copy()
X_resampled_with_target["PassOrFail"] = y_resampled
X_resampled_with_target.to_csv("./cn7_resampled.csv", index=False)


In [7]:
re = pd.read_csv("./cn7_test.csv")
re.info()
print(Xy_train["PassOrFail"].value_counts())
X_test_with_target["PassOrFail"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Injection_Time            1972 non-null   float64
 1   Filling_Time              1972 non-null   float64
 2   Plasticizing_Time         1972 non-null   float64
 3   Cycle_Time                1972 non-null   float64
 4   Clamp_Close_Time          1972 non-null   float64
 5   Cushion_Position          1972 non-null   float64
 6   Plasticizing_Position     1972 non-null   float64
 7   Clamp_Open_Position       1972 non-null   float64
 8   Max_Injection_Speed       1972 non-null   float64
 9   Max_Screw_RPM             1972 non-null   float64
 10  Average_Screw_RPM         1972 non-null   float64
 11  Max_Injection_Pressure    1972 non-null   float64
 12  Max_Switch_Over_Pressure  1972 non-null   float64
 13  Max_Back_Pressure         1972 non-null   float64
 14  Average_

PassOrFail
0    1957
1      15
Name: count, dtype: int64